#### Purpose Overview
The purpose of this script is to perform the quarterly target process, given updated prescribing behavior, payer contracts, interactions, Access, and territory Alignment info. The process takes historical data and uses a simple algorithm to categorize HCPs in buckets of importance, or target tiers. All HCPs are ranked according to a category of data (like AQ dispenses in the last six months) and the combination of the ranks on all categories produces a final rank. After the algorithm produces a final rank, the HCPs are sorted into buckets by territory and then submitted to the JTM process.


#### Inputs, Process, Outputs
The steps in this process are:
- 1 - Import relevant packages
- 2 - Define our weighting for our algorithm, and check that the weighting is balanced to 100%
- 3 - Define relevant data dates for the targeting process, and for additional JTM information. These dates are parameterized in our query and are updated quarterly.
- 4 - Connect to the warehouse and pull the relevant data for our foundational customer universe, then join both the current CU and JTM information together.
- 5 - Remove any exceptions that have been defined through partnership with the field. These are mainly HCPs who are high value, but do not interact with Kaleo.
- 6 - Join the most updated Zipterr and align territories with HCPs
- 7 - Join the newest data with our previous target list. This provides context in terms of what rankings were given originally to HCPs in the previous targeting process.
- 8 - Define functions to capture business logic like assigning calls to certain tiers
- 9 - Perform the ranking algorithm and apply business logic functions, clean file data for things like duplicates.
- 10 - Compare previous targeting process tier outputs with the current tiering to explore overlap of tiers
- 11 - Output xlsx and csv files of the guidance customer universe to be used for JTM
 

#### Tables or other resources referenced
- **commercial_analytics.mv_eai_dispense**
    - Used for dispense data. This will be replaced in a future iteration by domain.claim
    
- **commercial_analytics.mv_auviq_referral**
    - Used for referral data. This will be replaced in a future iteration by domain.claim
    
- **public.dim_hcp_entity**
    - Used for HCP detail data, has derivative detail tables
    
- **commercial_analytics.vw_mmit_koverage_overide_ops_targeting**
    - Used for coverage details for HCPs
    
 Please also see the sql files referenced in the query pull within this script - they can always be found within the current working directory where this script is located, in file labeled 'sql'.
    
##### Last Maintenance Date
- 6/06/2022

##### Future feature additions, updates
 - national rankings
 - update column name tier to suggested tier
 - update column name pre_jtm_calls to suggested calls

In [37]:
#load libraries

#for data processing and functions
from sqlalchemy import create_engine ## to connect to redshift
import pandas as pd #data handling
from pandas import ExcelWriter
import psycopg2 #for pulling data from redshift
import numpy as np #datahandling
from functools import reduce
import os
from sklearn.linear_model import LinearRegression

pd.set_option("max.columns", 200)


#### 2 - Variable inputs for Algorithm

In [38]:

rank_weighting = [.3,.05,.2,.2,.05,.2] # the weightings given to the ranking components of the targeting algorithm : 50% {EAI market, Market Share} 
                                    # 50% (AQ Packs, Expected Packs, Interactions, AQ packs yoy(qoq))
    
weighting_sum_check = sum(rank_weighting)
weighting_sum_check

if weighting_sum_check == 1.0:
    check_weight = True
else:
    check_weight = False

#### 3 - Relevant Dates for CU and JTM

In [39]:
# Full CU data

# dispense data spanning last 2 full quarters, dispense data specific to same quarter last year

dispense_date_sql_1 = '2021-10-01' 
dispense_date_sql_2 = '2022-03-31' # dispense data from the last full 2 quarters
dispense_date_sql_yoy_1 = '2021-07-01'
dispense_date_sql_yoy_2 = '2021-09-30' # dispense data from the same quarter previous year
claim_date_sql_1 = '2021-05-20'
claim_date_sql_2 = '2022-03-31' # claim data from the last full 4 quarters -- Issue with Asembia Data from before May 10 2021, No payer information - this data is for expected packs (our payer contracts)

claim_date_recent_sql_1 = '2022-01-01'
claim_date_recent_sql_2 = '2022-03-31' # claim data from the last full quarter
call_date_sql_1 = '2022-01-01'
call_date_sql_2 = '2022-03-31' # call data from the last full quarter

dates = {'date_1':dispense_date_sql_1,'date_2':dispense_date_sql_2,
         'date_3':claim_date_sql_1,'date_4':claim_date_sql_2,
         'date_5':claim_date_recent_sql_1,'date_6':claim_date_recent_sql_2,
         'date_7':call_date_sql_1,'date_8':call_date_sql_2,
         'date_9':dispense_date_sql_yoy_1,'date_10':dispense_date_sql_yoy_2,
        } # be sure to include a final comma in dictionary

In [40]:
# JTM Data

dispense_date_sql_jtm_1 = '2022-01-01' 
dispense_date_sql_jtm_2 = '2022-03-31' # dispense data from previous quarter
call_date_sql_jtm_1 = '2022-01-01'
call_date_sql_jtm_2 = '2022-03-31' # call data from previous quarter
dispense_date_sql_jtm_3 = '2021-07-01' 
dispense_date_sql_jtm_4 = '2021-09-30' # dispense data from the last year same time
call_date_sql_jtm_3 = '2021-07-01'
call_date_sql_jtm_4 = '2021-09-30' # call data from the last year same time


dates_jtm = {'date_1':dispense_date_sql_jtm_1,'date_2':dispense_date_sql_jtm_2,
         'date_3':call_date_sql_jtm_1,'date_4':call_date_sql_jtm_2,
         'date_5':dispense_date_sql_jtm_3,'date_6':dispense_date_sql_jtm_4,
         'date_7':call_date_sql_jtm_3,'date_8':call_date_sql_jtm_4,
            } # be sure to include a final comma in dictionary

#### Warehouse Connection

In [41]:
username = 'david_yolchuyev'
password = 'password'
host = 'host'
port = 'port'
db = 'database'


redshift_engine = create_engine("postgresql+psycopg2://{}:{}@{}:{}/{}".format(username, password, host, port, db), 
                                connect_args={'sslmode': 'prefer'})

#### 4 - Pull data from Warehouse

In [42]:
# Read sql file for CU data
script_dir = os.getcwd() # current working directory
rel_path = "sql\\targeting_sql_1.sql" # sql folder containing sql scripts - particular sql script needed
abs_file_path = os.path.join(script_dir, rel_path) # join current wd and sql script needed

query = open(abs_file_path, 'r') # open file with full path

with redshift_engine.connect() as conn, conn.begin():
    df = pd.read_sql_query(query.read(),conn, params = dates) # pandas dataframe from reading sql script


# Read sql file for JTM data
script_dir = os.getcwd() # current working directory
rel_path = "sql\\targeting_sql_2.sql" # sql folder containing sql scripts - particular sql script needed
abs_file_path = os.path.join(script_dir, rel_path) # join current wd and sql script needed

query = open(abs_file_path, 'r') # open file with full path

with redshift_engine.connect() as conn, conn.begin():
    df_jtm = pd.read_sql_query(query.read(),conn, params = dates_jtm) # pandas dataframe from reading sql script

query.close() 

# if you recieve error, it likely has to do with the parameterization - sql statements like "ilike '%auvi-q%'" must be "ilike '%%auvi-q%%'" because of python interaction with escape characters and params

#### Data Processing and pull-together

HCP data, dispense data for four quarters, claim data for four quarters, claim data from the most recent quarter, call data from the most recent quarter

In [43]:
df['npi_id'] = df['npi_id'].astype('int')
df_jtm['npi_id'] = df_jtm['npi_id'].astype('int')
df = df.drop_duplicates(subset=['npi_id']) # drop duplicate npis
df_jtm = df_jtm.drop_duplicates(subset=['npi_id']) # drop duplicate npis

In [44]:
cu = pd.merge(df, df_jtm, how='left', on = 'npi_id')
cu

,npi_id,profession_code,specialty_code,first_name,last_name,street_address_1,city,state,zipcode,prior_tier,claims,covered,coverage_perc,expected_packs,aq,comp,market,market_share,aq_yoy,comp_yoy,market_yoy,market_share_yoy,call_phone,call_fax,call_email,call_inperson,call_engage,total_calls,prev_q_aq,prev_q_market,prev_q_market_share,ly_aq,ly_market,ly_market_share,prev_q_call_inperson,ly_call_inperson
0,1831129261,MD,PDA,Luqman,Seidu,5671 Peachtree Dunwoody Ste 445,Atlanta,GA,30342,A,3099,847,0.273314,224.664085,1212.805,172.075,1384.880,0.8757,1186.0,169.765,1355.765,0.8747,5,0,4,8,0,17,708.0,792.0,0.8939,1186.0,1356.0,0.8746,8.0,1.0
1,1356339410,MD,AI,David,Fost,197 Bloomfield Ave,Verona,NJ,07044,A,1427,297,0.208129,82.419061,762.000,403.555,1165.555,0.6537,576.0,388.085,964.085,0.5974,0,0,0,13,0,13,414.0,592.0,0.6993,576.0,964.0,0.5975,13.0,0.0
2,1336109750,MD,AI,Amy,Darter,1810 E Memorial Rd Ste B,Oklahoma City,OK,73131,A,1264,204,0.161392,56.648734,448.000,158.755,606.755,0.7383,300.0,94.345,394.345,0.7607,0,0,1,18,0,19,186.0,259.0,0.7181,300.0,394.0,0.7614,18.0,8.0
3,1922075787,MD,AI,Allen,Lieberman,10801-2 N Mopac Expy Bldg 2-150,Austin,TX,78759,A,1243,394,0.316975,69.100563,698.000,90.840,788.840,0.8848,705.0,160.705,865.705,0.8143,2,0,2,11,1,16,313.0,347.0,0.9020,705.0,866.0,0.8140,11.0,11.0
4,1790753630,DO,AI,Tina,Zecca,200 White Rd Ste 205,Little Silver,NJ,07739,A,1068,280,0.262172,55.318352,491.000,221.270,712.270,0.6893,486.0,226.145,712.145,0.6824,4,0,0,20,0,24,247.0,369.0,0.6693,486.0,712.0,0.6825,20.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207303,1588688469,MD,PD,Laura,Weidenfeld,653 N Town Center Dr Ste 106,Las Vegas,NV,89144,B,0,0,0.000000,0.000000,0.000,5.935,5.935,0.0000,0.0,11.870,11.870,0.0000,0,0,0,4,0,4,0.0,3.0,0.0000,0.0,12.0,0.0000,4.0,7.0
207304,1497816995,MD,FM,Mildred,Frantz,1 Industrial Way W Bldg B,Eatontown,NJ,07724,B,0,0,0.000000,0.000000,0.000,15.105,15.105,0.0000,0.0,6.360,6.360,0.0000,0,0,0,0,0,0,0.0,8.0,0.0000,0.0,6.0,0.0000,0.0,0.0
207305,1497702872,MD,PD,Vidya,Krishnamurthy,3510 Old Milton Pkwy,Alpharetta,GA,30005,B,0,0,0.000000,0.000000,0.000,7.475,7.475,0.0000,0.0,11.570,11.570,0.0000,0,0,0,1,0,1,0.0,5.0,0.0000,0.0,12.0,0.0000,1.0,0.0
207306,1881603447,DO,PD,Beth,Hoff,14823 W Bell Rd Ste 208,Surprise,AZ,85374,B,0,0,0.000000,0.000000,0.000,12.380,12.380,0.0000,0.0,4.080,4.080,0.0000,0,0,0,0,0,0,0.0,3.0,0.0000,0.0,4.0,0.0000,0.0,0.0


In [45]:
# check_dupe_cg = cg.duplicated(subset=['npi_id'])
# ids = cg['npi_id']
# cg[ids.isin(ids[ids.duplicated()])]
# cg.drop_duplicates(subset=['npi_id'])

In [46]:
# df.to_csv('lm_test.csv')

#### 5 - Pre-ranking Exceptions Removal

In [47]:
pre_jtm_exceptions = pd.read_excel(r'C:\Users\griffin.salyer\OneDrive - KALEO, INC\Documents\Reports & Analysis\Targeting\2022 Q3\q3_2022_process\Final\targeting_exceptions_v1.xlsx', sheet_name='exceptions')
cu = cu[~cu.npi_id.isin(pre_jtm_exceptions)]

cu[cu['npi_id'] == '1619950250'] # HR exception 

,npi_id,profession_code,specialty_code,first_name,last_name,street_address_1,city,state,zipcode,prior_tier,claims,covered,coverage_perc,expected_packs,aq,comp,market,market_share,aq_yoy,comp_yoy,market_yoy,market_share_yoy,call_phone,call_fax,call_email,call_inperson,call_engage,total_calls,prev_q_aq,prev_q_market,prev_q_market_share,ly_aq,ly_market,ly_market_share,prev_q_call_inperson,ly_call_inperson


#### 6 - Zipterr join

In [48]:
zipterr = pd.read_excel(r'C:\Users\griffin.salyer\OneDrive - KALEO, INC\Documents\Reports & Analysis\Targeting\2022 Q3\q3_2022_process\Final\zipterr.xlsx', usecols = ['Zip','Territory_ID','Territory_Name','Area_ID','Area_Name'])
zipterr.head()
# need to update file location to cwdir for git

# <---Check - can create try except harness incase name for zip changes--->
zipterr['Zip'] = zipterr['Zip'].astype('str')

five_zip = []
for i in zipterr['Zip']:
    temp = i.zfill(5)
    five_zip.append(temp)
    
zipterr['Zip'] = five_zip

zipterr.rename({'Zip':'zipcode'}, axis =1, inplace = True)

check_dupe_zip = zipterr.duplicated(subset=['zipcode']).any()
# check_dupe_zip
# zipterr

#### 7 - Previous CU Join

In [49]:
cu = pd.merge(cu, zipterr, how='left', on = 'zipcode')
prev_cu = pd.read_csv(r'C:\Users\griffin.salyer\OneDrive - KALEO, INC\Documents\Reports & Analysis\Targeting\2022 Q3\q3_2022_process\Draft\q2_guidance_cu.csv', usecols=['npi_id','rank_final','tier'])
prev_cu.rename({'rank_final': 'rank_previous','tier':'previous_tier'}, axis=1, inplace=True)
prev_cu['npi_id'] = prev_cu['npi_id'].astype('int')
cu['npi_id'] = cu['npi_id'].astype('int')
cu = pd.merge(cu,prev_cu, how='left', on = 'npi_id')
# cu
# need to update file location to cwdir for git 

In [50]:
# process defined ONLY for Q3 2022 because of changing data infrastructure
# get access information on previous quarter targets, deprioritize targets to C tier
script_dir = os.getcwd() # current working directory
rel_path = "sql\\targeting_sql_4.sql" # sql folder containing sql scripts - particular sql script needed
abs_file_path = os.path.join(script_dir, rel_path) # join current wd and sql script needed

query = open(abs_file_path, 'r') # open file with full path

with redshift_engine.connect() as conn, conn.begin():
    q2_except = pd.read_sql_query(query.read(),conn) # pandas dataframe from reading sql script


In [51]:
cu = pd.merge(cu, q2_except, how='left', on = 'npi_id')
# cu[cu['target_tag_value'] == 'No Access – HCP will remain in the territory and may transfer to Virtual Sales.' ]
cu.fillna({'prev_q_aq':'0','prev_q_market':'0','prev_q_market_share':'0','ly_aq':'0','ly_market':'0','ly_market_share':'0',
           'prev_q_call_inperson':'0','ly_call_inperson':'0','previous_tier':'NT'},inplace=True) #fill NaN values in these columns to fill in voumes for NPIs who did not have transactional ->
                    #records in the database for the defined time periods
cu

,npi_id,profession_code,specialty_code,first_name,last_name,street_address_1,city,state,zipcode,prior_tier,claims,covered,coverage_perc,expected_packs,aq,comp,market,market_share,aq_yoy,comp_yoy,market_yoy,market_share_yoy,call_phone,call_fax,call_email,call_inperson,call_engage,total_calls,prev_q_aq,prev_q_market,prev_q_market_share,ly_aq,ly_market,ly_market_share,prev_q_call_inperson,ly_call_inperson,Territory_ID,Territory_Name,Area_ID,Area_Name,rank_previous,previous_tier,mandatory reason code
0,1831129261,MD,PDA,Luqman,Seidu,5671 Peachtree Dunwoody Ste 445,Atlanta,GA,30342,A,3099,847,0.273314,224.664085,1212.805,172.075,1384.880,0.8757,1186.0,169.765,1355.765,0.8747,5,0,4,8,0,17,708,792,0.8939,1186,1356,0.8746,8,1,QS0202,"Atlanta North, GA",QS02,South Atlantic,1.0,A,NaN
1,1356339410,MD,AI,David,Fost,197 Bloomfield Ave,Verona,NJ,07044,A,1427,297,0.208129,82.419061,762.000,403.555,1165.555,0.6537,576.0,388.085,964.085,0.5974,0,0,0,13,0,13,414,592,0.6993,576,964,0.5975,13,0,QN0206,"Jersey City, NJ",QN02,Metro New York,2.0,A,NaN
2,1336109750,MD,AI,Amy,Darter,1810 E Memorial Rd Ste B,Oklahoma City,OK,73131,A,1264,204,0.161392,56.648734,448.000,158.755,606.755,0.7383,300.0,94.345,394.345,0.7607,0,0,1,18,0,19,186,259,0.7181,300,394,0.7614,18,8,QS0414,"Oklahoma City, OK",QS04,Great Plains,1.0,A,NaN
3,1922075787,MD,AI,Allen,Lieberman,10801-2 N Mopac Expy Bldg 2-150,Austin,TX,78759,A,1243,394,0.316975,69.100563,698.000,90.840,788.840,0.8848,705.0,160.705,865.705,0.8143,2,0,2,11,1,16,313,347,0.902,705,866,0.814,11,11,QS0311,"Austin, TX",QS03,South Central,1.0,A,NaN
4,1790753630,DO,AI,Tina,Zecca,200 White Rd Ste 205,Little Silver,NJ,07739,A,1068,280,0.262172,55.318352,491.000,221.270,712.270,0.6893,486.0,226.145,712.145,0.6824,4,0,0,20,0,24,247,369,0.6693,486,712,0.6825,20,18,QN0301,"New Brunswick, NJ",QN03,Mid Atlantic,1.0,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207303,1588688469,MD,PD,Laura,Weidenfeld,653 N Town Center Dr Ste 106,Las Vegas,NV,89144,B,0,0,0.000000,0.000000,0.000,5.935,5.935,0.0000,0.0,11.870,11.870,0.0000,0,0,0,4,0,4,0,3,0,0,12,0,4,7,QS0610,"Las Vegas, NV",QS06,Rocky Mountains,103.0,B,NaN
207304,1497816995,MD,FM,Mildred,Frantz,1 Industrial Way W Bldg B,Eatontown,NJ,07724,B,0,0,0.000000,0.000000,0.000,15.105,15.105,0.0000,0.0,6.360,6.360,0.0000,0,0,0,0,0,0,0,8,0,0,6,0,0,0,QN0301,"New Brunswick, NJ",QN03,Mid Atlantic,133.0,B,NaN
207305,1497702872,MD,PD,Vidya,Krishnamurthy,3510 Old Milton Pkwy,Alpharetta,GA,30005,B,0,0,0.000000,0.000000,0.000,7.475,7.475,0.0000,0.0,11.570,11.570,0.0000,0,0,0,1,0,1,0,5,0,0,12,0,1,0,QS0202,"Atlanta North, GA",QS02,South Atlantic,172.0,B,NaN
207306,1881603447,DO,PD,Beth,Hoff,14823 W Bell Rd Ste 208,Surprise,AZ,85374,B,0,0,0.000000,0.000000,0.000,12.380,12.380,0.0000,0.0,4.080,4.080,0.0000,0,0,0,0,0,0,0,3,0,0,4,0,0,0,QS0611,"Phoenix, AZ",QS06,Rocky Mountains,143.0,B,NaN


#### 8 - Function Definitions

In [52]:
def apply_prior_target(row):
    if (row['previous_tier'] == 'A') | (row['previous_tier'] == 'B') | (row['previous_tier'] == 'C'): # change to terr_tier?????
        return True
    else: 
        return False

def apply_deprioritize(row): # deprioritizes specified specialties - We dont want to call on emergency medicine, for example
    if (row['specialty_code'] == 'EFM') | (row['specialty_code'] == 'EM') | (row['specialty_code'] == 'EMP') |  (row['specialty_code'] == 'EMS'):
        return 400000
    else:
        return row['rank_non_pri']
    
def apply_deprioritize_access(row): # deprioritizes specified specialties - We dont want to call on emergency medicine, for example
    if (row['mandatory reason code'] == 'No Access – HCP will remain in the territory and may transfer to Virtual Sales.'):
        return 400000
    else:
        return row['rank_non_pri']
    
def apply_tier(row): # create our target tiers according to tier allocation logic
    if row['rank_final'] <= 30 :
        return 'A'
    elif (row['rank_final'] > 30) & (row['rank_final'] <= 200) :
        return 'B'
    else:
        return 'C'

def remove_terr_nans(df): # remove any blank territories
    df['Territory_ID'].replace('', np.nan, inplace=True)
    df.dropna(subset=['Territory_ID'], inplace=True)
    df.columns = [x.lower() for x in df.columns] # make all column names lowercase
    return df

def apply_calls(row): # apply call alocation logic based on business criteria
    if row['rank_final'] <= 20 :
        return '5'
    elif (row['rank_final'] > 20) & (row['rank_final'] <= 30) :
        return '4'
    elif (row['rank_final'] > 30) & (row['rank_final'] <= 75) :
        return '3'
    elif (row['rank_final'] > 75) & (row['rank_final'] <= 100) :
        return '2'
    elif (row['rank_final'] > 100) & (row['rank_final'] <= 200) :
        return '1'
    else:
        return '0'

#### 9 - Ranking and Algorithms and function application

In [53]:
rank_method = 'first'

cu['rank_eai_market'] = cu.groupby('Territory_ID')['market'].rank(rank_method, ascending=False)
cu = cu.sort_values(['rank_eai_market'])
cu['rank_aq'] = cu.groupby('Territory_ID')['aq'].rank(rank_method, ascending=False)
cu = cu.sort_values(['rank_eai_market','rank_aq'])
cu['rank_aq_yoy'] = cu.groupby('Territory_ID')['aq_yoy'].rank(rank_method, ascending=False)
cu = cu.sort_values(['rank_eai_market','rank_aq','rank_aq_yoy'])

cu['rank_ms'] = cu.groupby('Territory_ID')['market_share'].rank(rank_method, ascending=False)
cu['rank_expected_packs'] = cu.groupby('Territory_ID')['expected_packs'].rank(rank_method, ascending=False)
cu['rank_interactions'] = cu.groupby('Territory_ID')['total_calls'].rank(rank_method, ascending=False)


cu['weighted_avg_score'] = (cu['rank_eai_market']*rank_weighting[0] + cu['rank_ms']*rank_weighting[1] + cu['rank_aq']*rank_weighting[2] + \
                        cu['rank_expected_packs']*rank_weighting[3] + cu['rank_interactions']*rank_weighting[4] + cu['rank_aq_yoy']*rank_weighting[5])




cu['rank_non_pri'] = cu.groupby('Territory_ID')['weighted_avg_score'].rank("first", ascending=True)

cu['rank_non_pri'] = cu.apply(apply_deprioritize_access, axis =1)
cu['rank_non_pri'] = cu.apply(apply_deprioritize, axis =1)

cu['rank_final'] = cu.groupby('Territory_ID')['rank_non_pri'].rank("first", ascending=True)

# cu

In [54]:
cu['prior_tier'] = cu.apply(apply_prior_target, axis = 1)
cu['tier'] = cu.apply(apply_tier, axis=1)
cu = remove_terr_nans(cu)
cu['pre_jtm_calls'] = cu.apply(apply_calls, axis=1)
# cu
cu[cu['npi_id'] == '1619950250']
# Development - need to check NaN values and ensure they're operating as expected for those columns

,npi_id,profession_code,specialty_code,first_name,last_name,street_address_1,city,state,zipcode,prior_tier,claims,covered,coverage_perc,expected_packs,aq,comp,market,market_share,aq_yoy,comp_yoy,market_yoy,market_share_yoy,call_phone,call_fax,call_email,call_inperson,call_engage,total_calls,prev_q_aq,prev_q_market,prev_q_market_share,ly_aq,ly_market,ly_market_share,prev_q_call_inperson,ly_call_inperson,territory_id,territory_name,area_id,area_name,rank_previous,previous_tier,mandatory reason code,rank_eai_market,rank_aq,rank_aq_yoy,rank_ms,rank_expected_packs,rank_interactions,weighted_avg_score,rank_non_pri,rank_final,tier,pre_jtm_calls


In [19]:
# cu

#### 10 - Comparisons

In [56]:
# cg_2 = cg[cg['Territory_ID'] != 'Q99999']

## Compare As
compare_df = cu[cu['tier'] == 'A']
count_out = compare_df[compare_df['tier'] == compare_df['previous_tier']]['npi_id'].count()
count_in = compare_df['npi_id'].count()
compare_a = count_out/count_in
# A.append(compare_a)

## Compare Bs
compare_df = cu[cu['tier'] == 'B']
count_out = compare_df[compare_df['tier'] == compare_df['previous_tier']]['npi_id'].count()
count_in = compare_df['npi_id'].count()
compare_b = count_out/count_in
# B.append(compare_b)

## Compare ABs
compare_df = cu.loc[(cu['tier'] == 'A') | (cu['tier'] == 'B')] 
count_out = compare_df[compare_df['tier'] == compare_df['previous_tier']]['npi_id'].count()
count_in = compare_df['npi_id'].count()
compare_ab = count_out/count_in
# AB.append(compare_ab)

## Compare 200
prev_count = cu[cu['rank_previous'] <= 200]
current_count = cu[cu['rank_final'] <= 200]
count_in = current_count['npi_id'].count() 
merge_200 = pd.merge(current_count,prev_count, how = 'inner', on = 'npi_id')
count_out = merge_200['npi_id'].count()
compare_200 = count_out/count_in
# _200.append(compare_200)

print(compare_a)
print(compare_b)
print(compare_ab)
print(compare_200)

0.8766025641025641
0.6735294117647059
0.7039903846153847
0.7351923076923077


In [22]:
customer_universe = cu

In [23]:
# reduce the unnecessary columns
# drop_cols = ['prior_tier','claims','covered',
#              'coverage_perc','expected_packs','aq',
#              'comp','market','market_share','call_phone',
#             'call_fax','call_email','call_inperson','call_engage',
#              'total_calls','rank_eai_market','rank_aq','rank_ms',
#              'rank_expected_packs','rank_interactions','weighted_avg_score',
#              'rank_non_pri','terr_rank','terr_tier']
# customer_universe = customer_universe.drop(drop_cols, axis=1, errors='ignore')

In [44]:
# customer_universe

#### Guidance Customer Galaxy

#### 11 - Output Guidance CU

In [27]:
# if check_weight == False:
#     print('check_weight error')
#     raise SystemExit()
# elif check_dupe_zip == True:
#     print('check_dupe_zip error')
#     raise SystemExit()    
# else:
#     print('No error, writing out DF')
#     cu4.to_csv('targeting_cu_v2.csv', index= False)
#     cu4.to_excel('targeting_cu_v2.xlsx', index = False)

In [55]:
cu.to_csv(r'C:\Users\david\targeting_guidance_cu.csv', index= False)

#### Warehouse Output

#### Comparisons and analytics

In [268]:
# prev_cu.rename({'npi':'npi_id'},axis=1,inplace=True)
# cu4['npi_id'] = cu4['npi_id'].astype('int') 
# compare_cu = pd.merge(cu4,prev_cu, how='left', on ='npi_id')
# compare_cu.to_excel('test_cu.xlsx', index = False)

In [192]:
list_of_weights = []
n = 11
for i in range(1,n):
    a = np.random.random(5)
    a /= a.sum()
    list_of_weights.append(a.tolist())

weights_df = pd.DataFrame(list_of_weights, columns = ['EAI','MS','AQ','Expected Packs','Interactions'])

A = []
B = []
AB = []
_200 = []

def compare_weights(list_of_weights):

    for rank_weighting in list_of_weights:

        cu4['weighted_avg_score'] = (cu4['rank_eai_market']*rank_weighting[0] + cu4['rank_ms']*rank_weighting[1] + cu4['rank_aq']*rank_weighting[2] + \
                                cu4['rank_expected_packs']*rank_weighting[3] + cu4['rank_interactions']*rank_weighting[4])
        cu4['rank_final'] = cu4.groupby('Territory_ID')['weighted_avg_score'].rank("first", ascending=True)

        cu4['tier'] = cu4.apply(apply_tier, axis=1)

        ## Compare As
        compare_df = cu4[cu4['tier'] == 'A']
        count_out = compare_df[compare_df['tier'] == compare_df['terr_tier']]['npi_id'].count()
        count_in = compare_df['npi_id'].count()
        compare_a = count_out/count_in
        A.append(compare_a)

        ## Compare Bs
        compare_df = cu4[cu4['tier'] == 'B']
        count_out = compare_df[compare_df['tier'] == compare_df['terr_tier']]['npi_id'].count()
        count_in = compare_df['npi_id'].count()
        compare_b = count_out/count_in
        B.append(compare_b)

        ## Compare ABs
        compare_df = cu4.loc[(cu4["tier"] == 'A') | (cu4["tier"] == 'B')] 
        count_out = compare_df[compare_df['tier'] == compare_df['terr_tier']]['npi_id'].count()
        count_in = compare_df['npi_id'].count()
        compare_ab = count_out/count_in
        AB.append(compare_ab)

        ## Compare 200
        prev_count = cu4[cu4['terr_rank'] <= 200]
        current_count = cu4[cu4['rank_final'] <= 200]
        count_in = current_count['npi_id'].count() 
        merge_200 = pd.merge(current_count,prev_count, how = 'inner', on = 'npi_id')
        count_out = merge_200['npi_id'].count()
        compare_200 = count_out/count_in
        _200.append(compare_200)

In [193]:
# list_1 = []
# n = 100
# for i in range(1,n):
#     list_1.append((np.random.dirichlet(np.ones(5),size=1)).tolist())
# list_of_weights  = [val for sublist in list_1 for val in sublist]



rank_method = 'first'

cu4['rank_eai_market'] = cu4.groupby('Territory_ID')['market'].rank(rank_method, ascending=False)
cu4 = cu4.sort_values(['rank_eai_market'])
cu4['rank_aq'] = cu4.groupby('Territory_ID')['aq'].rank(rank_method, ascending=False)
cu4 = cu4.sort_values(['rank_eai_market','rank_aq'])

cu4['rank_ms'] = cu4.groupby('Territory_ID')['market_share'].rank(rank_method, ascending=False)
cu4['rank_expected_packs'] = cu4.groupby('Territory_ID')['expected_packs'].rank(rank_method, ascending=False)
cu4['rank_interactions'] = cu4.groupby('Territory_ID')['total_calls'].rank(rank_method, ascending=False)




compare_weights(list_of_weights)
overlap_df = pd.DataFrame({'A':A,'B':B,'AB':AB,'_200':_200})
dfs = [weights_df,overlap_df]

df = pd.concat(dfs, axis = 1)


In [194]:
df

,EAI,MS,AQ,Expected Packs,Interactions,A,B,AB,_200
0,0.332435,0.019345,0.163537,0.268651,0.216032,0.811006,0.527012,0.569817,0.754337
1,0.151967,0.451813,0.207055,0.074155,0.115010,0.668239,0.516799,0.539625,0.790975
2,0.222711,0.209808,0.072834,0.266985,0.227663,0.760063,0.527124,0.562233,0.767514
3,0.241922,0.040331,0.241553,0.146447,0.329747,0.795597,0.539290,0.577922,0.767371
4,0.023818,0.290164,0.202114,0.190636,0.293267,0.693396,0.530528,0.555076,0.789506
5,0.309674,0.361419,0.248460,0.026295,0.054151,0.721384,0.518696,0.549246,0.780643
6,0.164507,0.406763,0.015197,0.192784,0.220749,0.688365,0.522715,0.547682,0.779458
7,0.019763,0.264059,0.152374,0.282100,0.281704,0.697799,0.526733,0.552517,0.784198
8,0.327118,0.207581,0.012201,0.087204,0.365895,0.762579,0.528575,0.563845,0.756470
9,0.287847,0.118132,0.273333,0.050452,0.270236,0.793082,0.537281,0.575837,0.770263


In [65]:
df.to_csv('weights_test_v4.csv')

In [24]:
## Compare As
compare_df = cu[cu['tier'] == 'A']
count_out = compare_df[compare_df['tier'] == compare_df['terr_tier']].groupby('territory_id')['npi_id'].count()
count_in = compare_df.groupby('territory_id')['npi_id'].count()
compare_a = (count_out/count_in).reset_index()
print(compare_a[compare_a['npi_id'] == max(compare_a['npi_id'])])
print(compare_a[compare_a['npi_id'] == min(compare_a['npi_id'])])
# compare_a.to_csv('compare_a_terr.csv')

   territory_id    npi_id
49       QN0607  0.966667
    territory_id  npi_id
100       QS0605     0.6


In [25]:
## Compare 200
prev_count = cu[cu['terr_rank'] <= 200]
current_count = cu[cu['rank_final'] <= 200]
count_in = current_count.groupby('territory_id')['npi_id'].count() 
merge_200 = pd.merge(current_count,prev_count, how = 'inner', on = ['npi_id','territory_id'])
count_out = merge_200.groupby('territory_id')['npi_id'].count()
compare_200 = (count_out/count_in).reset_index()
print(compare_200[compare_200['npi_id'] == max(compare_200['npi_id'])])
print(compare_200[compare_200['npi_id'] == min(compare_200['npi_id'])])
# compare_200.to_csv('compare_200_terr.csv')

   territory_id  npi_id
61       QS0201   0.885
   territory_id  npi_id
25       QN0309    0.62


In [26]:
compare_df = cu[cu['tier'] == 'A']
print(np.mean(compare_df['market']))
print(np.mean(compare_df['market_share']))
print(np.mean(compare_df['aq']))
print(np.mean(compare_df['expected_packs']))
print(np.mean(compare_df['total_calls']))
print('')
print(np.median(compare_df['market']))
print(np.median(compare_df['market_share']))
print(np.median(compare_df['aq']))
print(np.median(compare_df['expected_packs']))
print(np.median(compare_df['total_calls']))

290.68248412698364
0.47998608623799816
138.38495079365097
13.832089989570251
5.894603174603175

213.10750000000002
0.47593951421620995
89.0
8.333333333333334
5.0


In [27]:
compare_df = cu[cu['tier'] == 'B']
print(np.mean(compare_df['market']))
print(np.mean(compare_df['market_share']))
print(np.mean(compare_df['aq']))
print(np.mean(compare_df['expected_packs']))
print(np.mean(compare_df['total_calls']))

40.067185154061825
0.1842050473042479
6.655020728291315
0.7613506344924433
1.6975350140056022


In [28]:
compare_df = cu[cu['tier'] == 'C']
print(np.mean(compare_df['market']))
print(np.mean(compare_df['market_share']))
print(np.mean(compare_df['aq']))
print(np.mean(compare_df['expected_packs']))
print(np.mean(compare_df['total_calls']))

4.126678719214276
0.0161097941733821
0.07272056841310705
0.008289226935647332
0.02816118292469234
